# BRIDGE CROSSING PROBLEM

October 12, 2017

Welcome! This IPython notebook will go through the implementation of the the Uniform Cost Search Algorithm and A\* Search Algorithm to solve the famous Bridge Crossing Problem. An analysis and comparison of the two algorithms will also be shown.

# 1. Search Algorithms

The Task Environment for the search algorithms discussed below is given by:


### TASK_ENVIRONMENT =

    Performance Measure: Shows the minimal cost for solving the Bridge Crossing Problem, the number of nodes
    visited, and the sequence of steps for solving the problem

    External Environment: Terminal application (e.g. Command Prompt)

    Actuators: Text-based output to a terminal

    Sensors: List of integers from user input

The state of the environment is represented as a list of boolean values with the *nth* element of the list corresponding to the *nth* person on the environment. The value of 0 means that the *nth* person has not crossed the bridge (at the left of the bridge), while the value of 1 means that the *nth* person crossed the bridge (at the right of the bridge). The state $[0,0,0,...,0]$ is an initial state, while $[1,1,1,...,1]$ is the goal state. An example is shown below.

*e.g.* Given 4 people, the state $[1,0,0,1]$ means that the *1st* and *4th* person already crossed, and the *2nd* and *3rd* hasn't.

The state space, on the other hand, is represented as a tree. The tree has multiple nodes and multiple branches. Nodes contain useful information about the current state of the environment, while the branches connect two nodes and it has a cost corresponding to the cost of going from the initial state to the current state.

## 1.1 Uniform Cost Search

Useful libraries are imported. Bisect is used in this project for optimally inserting elements in a list in a sorted manner, while Itertools is used to generate a combination of a list.

In [11]:
import bisect, itertools

To reduce code complexity, the nodes of a tree are objects with cost, state, and steps as attributes.

*COST (Integer) -->* cost of going from the initial state to current state.<br />
*STATE (List) -->* the current state of the environment as defined above.<br />
*STEPS (List) -->* the sequence of steps of the people leading to the current state.<br />

*e.g.* Given a Node $(10, [1,0,0,0], [1,3,3])$ means that the cost from initial state to current state, $[1,0,0,0]$, is 10. Moreover, the sequence of steps from initial state to current state is $[1,3,3]$, which means that 1 and 3 has crossed the bridge and 3 went back with the flashlight.

Notice that the inclusion of the steps attribute will aid with the identification of the path from the initial state to the goal state. With this, the sequence of steps of the solution can easily be printed out. The function print_steps uses the steps attribute to print the sequence of steps.

In [12]:
class Node:
    def __init__(self, cost, state, steps):
        self.cost = cost
        self.state = state
        self.steps = steps

    def __lt__(self, other):
        return self.cost < other.cost

    def print_steps(self):
        i = 0
        while i < len(self.steps):
            if (i+1) % 3 == 0:
                print(self.steps[i], '<-')
                i += 1
            else:
                print(self.steps[i], self.steps[i+1], '->')
                i += 2

<br />The code snippet below is the Tree class. This is the code representation of the state space. The class contains the following attributes:

*FRINGE (List) -->* collection of nodes yet to be expanded.<br />
*FITNESS (List) -->* the original input; a list of the time required for the people to cross the bridge.

The fringe initially contains the root node, which contains the initial state, a cost of 0 (since it is the root node), and an empty steps list. The different class methods will be explained after the code snippet below.

In [13]:
class Tree:
    def __init__(self, fitness):
        state = [0] * len(fitness)
        steps = []
        cost = 0
        root = Node(cost, state, steps)

        self.fringe = [root]
        self.fitness = fitness

    def has_repeated_states(self, node):
        for item in self.fringe:
            if node.state == item.state and node.cost < item.cost:
                self.fringe.remove(item)
                bisect.insort(self.fringe, node)
                return True
            if node.state == item.state:
                return True
        return False

    def generate_nodes(self, node):
        fitness_len = len(self.fitness)
        if len(node.steps) % 3 == 0:
            for elem in itertools.combinations(list(range(fitness_len)), 2):
                if (node.state[elem[0]] == 0 and node.state[elem[1]] == 0):
                    new_state = list(node.state)
                    new_state[elem[0]] = 1
                    new_state[elem[1]] = 1

                    new_steps = list(node.steps)
                    new_steps.extend([elem[0]+1, elem[1]+1])

                    new_cost = node.cost + max(self.fitness[elem[0]], self.fitness[elem[1]])

                    new_node = Node(new_cost, new_state, new_steps)

                    # Check for Repeated States
                    if not self.has_repeated_states(new_node):
                        bisect.insort(self.fringe, new_node)

        else:
            for elem in list(range(fitness_len)):
                if (node.state[elem] == 1):
                    new_state = list(node.state)
                    new_state[elem] = 0

                    new_steps = list(node.steps)
                    new_steps.append(elem+1)

                    new_cost = node.cost + self.fitness[elem]

                    new_node = Node(new_cost, new_state, new_steps)

                    # Check for Repeated States
                    if not self.has_repeated_states(new_node):
                        bisect.insort(self.fringe, new_node)

    def uniform_cost_search(self):
        num_visited = 0
        fitness_len = len(self.fitness)
        while True:
            if not self.fringe:
                return

            # Explore Fringe
            node = self.fringe.pop(0)
            num_visited += 1

            # Check for Goal State
            if node.state == [1] * fitness_len:
                print(node.cost, num_visited)
                node.print_steps()
                return

            # Generate Successor States
            self.generate_nodes(node)

<br />This class method contains the Uniform Cost Search algorithm. It continues to loop until either a goal state is found or if there exists no goal state. If the fringe is empty, then there exists no goal state. The algorithm continues to pop a node from the fringe and adds the successor nodes of that node inside the fringe while keeping the fringe sorted. As a modification to the algorithm, a loop counter was placed to identify the number of expanded nodes.

The successor nodes are generated by calling the class method generate_nodes with an argument of the popped node.

In [14]:
def uniform_cost_search(self):
        num_visited = 0
        fitness_len = len(self.fitness)
        while True:
            if not self.fringe:
                return

            # Explore Fringe
            node = self.fringe.pop(0)
            num_visited += 1

            # Check for Goal State
            if node.state == [1] * fitness_len:
                print(node.cost, num_visited)
                node.print_steps()
                return

            # Generate Successor States
            self.generate_nodes(node)

<br />The generate_nodes class method generates all possible successor nodes from the passed node argument. The usual argument being passed into this method is the recently popped node from the fringe.

In the search space, there are two types of state being generated: a state where two people cross the bridge, and a state where one person returns with the flashlight. Given this configuration, how is the type of a state being identified? This greatly utilizes the node attribute: node.steps. Notice that whenever the length of node.steps is a multiple of 3, then the recent move was a person returning with the flashlight. Therefore, the next move should be two people crossing the bridge. Likewise, if the length of node.steps is not a multiple of 3, then the recent move was two people crossing. Therefore, the next move should be a person crossing back.

*e.g.* Given node.steps $[1,3,1,4,2]$ means that 1 and 3 crossed the bridge, 1 returned, then 4 and 2 crossed the bridge. The length of this is not a multiple of 3, hence all possible successor states from this are states where one person crosses back with the flashlight.

For the type of state where two people cross the bridge, the itertools library was used. The library was used to generate all possible 2-Combination of fitness_len. This returns a list of 2-tuple which then was used to generate the successor nodes. The numbers inside the 2-tuple are basically the two people crossing the bridge. The node.step is updated by changing 0s to a 1 (note that it is first checked if the two people have not crossed the bridge yet). The two people are also appended to the end of the node.steps. Lastly, node.cost is updated by adding the argument node's cost (path cost of the root to the parent node) with the max of the fitness of the two people.

The same principle happens with the type of state where one person returns. All possible successor nodes are generated (there is no need to use itertools here, a simple loop is enough): node.step is updated by changing 1s to a 0, the person gets appended to the end of node.step, and node.cost is updated by adding parent node's cost with the fitness of that person.

Lastly, repeated states must be checked first before the newly generated node gets inserted into the fringe.

In [15]:
def generate_nodes(self, node):
        fitness_len = len(self.fitness)
        if len(node.steps) % 3 == 0:
            for elem in itertools.combinations(list(range(fitness_len)), 2):
                if (node.state[elem[0]] == 0 and node.state[elem[1]] == 0):
                    new_state = list(node.state)
                    new_state[elem[0]] = 1
                    new_state[elem[1]] = 1

                    new_steps = list(node.steps)
                    new_steps.extend([elem[0]+1, elem[1]+1])

                    new_cost = node.cost + max(self.fitness[elem[0]], self.fitness[elem[1]])

                    new_node = Node(new_cost, new_state, new_steps)

                    # Check for Repeated States
                    if not self.has_repeated_states(new_node):
                        bisect.insort(self.fringe, new_node)

        else:
            for elem in list(range(fitness_len)):
                if (node.state[elem] == 1):
                    new_state = list(node.state)
                    new_state[elem] = 0

                    new_steps = list(node.steps)
                    new_steps.append(elem+1)

                    new_cost = node.cost + self.fitness[elem]

                    new_node = Node(new_cost, new_state, new_steps)

                    # Check for Repeated States
                    if not self.has_repeated_states(new_node):
                        bisect.insort(self.fringe, new_node)

<br />The has_repeated_states class method checks if the state of a node (node.state) is already contained within the fringe. This is a crucial part of the Uniform Cost Search algorithm since a repeated state is to be avoided. The idea is to replace a node in a fringe with another node with the same state but with a cheaper cost. If node.state is in the fringe, the path cost to the node has to be checked. If it is less than the path cost of the node stored in the fringe, the node in the fringe has to be replaced. If it is not, nothing has to be done.

The has_repeated_states class method takes in a node as a parameter. The usual argument being passed into this method is a newly generated node from the generate_nodes method. The bisect library was also used to insort nodes into the fringe while keeping the fringe sorted.

In [16]:
def has_repeated_states(self, node):
        for item in self.fringe:
            if node.state == item.state and node.cost < item.cost:
                self.fringe.remove(item) # CHANGE this to remove(index)
                bisect.insort(self.fringe, node)
                return True
            if node.state == item.state:
                return True
        return False

## 1.2 A\* Search

First, we'll define a new class that is similar to the Node class written above. Note that we added a new attribute, evaluation_function, which equals the cost + heuristic. The evaluation_function attribute will be used to compare nodes to know which ones are cheaper instead of the cost attribute that was used earlier in Uniform Cost Search. The print_steps method is the same as before.

In [17]:
class Node_A_star:
    def __init__(self, cost, heuristic, state, steps):
        self.cost = cost
        self.evaluation_function = cost + heuristic
        self.state = state
        self.steps = steps

    def __lt__(self, other):
        return self.evaluation_function < other.evaluation_function

    def print_steps(self):
        i = 0
        while i < len(self.steps):
            if (i+1) % 3 == 0:
                print(self.steps[i], '<-')
                i += 1
            else:
                print(self.steps[i], self.steps[i+1], '->')
                i += 2

Next, we'll define the Tree class that will be used for the A\* Search. We can notice a few changes as well.

In [28]:
class Tree_A_star:
    def __init__(self, fitness):
        state = [0] * len(fitness)
        steps = []
        cost = 0
        root = Node_A_star(cost, 0, state, steps)

        self.orig = fitness
        self.fringe = [root]
        self.fitness = sorted(fitness)

    def has_repeated_states(self, node):
        for item in self.fringe:
            if node.state == item.state and node.evaluation_function < item.evaluation_function:
                self.fringe.remove(item)
                bisect.insort(self.fringe, node)
                return True
            if node.state == item.state:
                return True
        return False
    
    def find_heuristic(self,node,place):
        l = []
        h = 0
        for elem in list(range(len(self.fitness))):
                if node.state[elem] == place:
                        l.append(self.fitness[elem])
        l = l[::-1]
        for elem in range(len(l),2):
                h += l[elem]
        return h
    
    def generate_nodes(self,node):
        fitness_len = len(self.fitness)
        if len(node.steps) % 3 == 0:
            h = self.find_heuristic(node,0)
            fin = []
            left = []
            c = 0
            for elem in range(fitness_len):
                    if node.state[elem] == 0:
                            left.append(elem)
                            c += 1
            a = left[0]
            b = left[1]           
            if c == 2:
                fin = [[a,b]]                
            else:
                c = left[-1]
                d = left[-2]
                fin = [[a,b],[a,c],[d,c]]
               
            c = 0
            for elem in fin:
                    new_state = list(node.state)
                    new_state[elem[0]] = 1
                    new_state[elem[1]] = 1
                    
                    new_steps = list(node.steps)
                    if(self.fitness[elem[0]]==self.fitness[elem[1]]):
                        for f in range(self.orig.index(self.fitness[elem[0]])+1,fitness_len):
                            if(self.fitness[elem[1]]==self.fitness[f]):
                                new_steps.extend([self.orig.index(self.fitness[elem[0]])+1, f+1])
                                break
                    else:
                        new_steps.extend([self.orig.index(self.fitness[elem[0]])+1, self.orig.index(self.fitness[elem[1]])+1])

                    new_cost = node.cost + max(self.fitness[elem[0]], self.fitness[elem[1]])
                    new_node = Node_A_star(new_cost, h , new_state, new_steps)
                    
                    if not self.has_repeated_states(new_node):
                            bisect.insort(self.fringe, new_node)
        else:
            h = self.find_heuristic(node,1)
            for elem in list(range(fitness_len)):
                if (node.state[elem] == 1):
                    new_state = list(node.state)
                    new_state[elem] = 0

                    new_steps = list(node.steps)
                    new_steps.append(self.orig.index(self.fitness[elem])+1)

                    new_cost = node.cost + self.fitness[elem]

                    new_node = Node_A_star(new_cost, h, new_state, new_steps)
                    
                    if not self.has_repeated_states(new_node):
                        bisect.insort(self.fringe, new_node)
                        
    def a_star(self):
        num_visited = 0;
        fitness_len = len(self.fitness)
        while True:
            if not self.fringe:
                return 'Failure'

            # Explore Fringe
            node = self.fringe.pop(0)
            num_visited += 1
            if node.state == [1] * fitness_len:
                print(node.cost, num_visited)
                node.print_steps()
                return 'Success'

            self.generate_nodes(node)

The orig attribute will be used to keep track of the original indices of the input because we'll want the list sorted for later.

In [19]:
class Tree_A_star:
    def __init__(self, fitness):
        state = [0] * len(fitness)
        steps = []
        cost = 0
        root = Node_A_star(cost, 0, state, steps)

        self.orig = fitness
        self.fringe = [root]
        self.fitness = sorted(fitness)

The has_repeated_states now uses the evaluation_function instead of the cost as described above.

In [20]:
def has_repeated_states(self, node):
    for item in self.fringe:
        if node.state == item.state and node.evaluation_function < item.evaluation_function:
            self.fringe.remove(item)
            bisect.insort(self.fringe, node)
            return True
        if node.state == item.state:
            return True
    return False

We now look at our heuristic function. Here, we check where the flashlight is located with the place variable (it is 0 if the flashlight is on the initial side and 1 if it is on the goal side). Then, we take all persons on the selected side (the people that can cross the bridge) and store them in a list. This is where the list being sorted comes into play. We reverse the order of the sorted list so that the first element will be greater than or equal to the next item in the list and so on (i.e. c0 >= c1 >= c2 ... >= cn) and add up all even indexed elements in the list and return that value. 

In [21]:
def find_heuristic(self,node,place):
        l = []
        h = 0
        for elem in list(range(len(self.fitness))):
                if node.state[elem] == place:
                        l.append(self.fitness[elem])
        l = l[::-1]
        for elem in range(len(l),2):
                h += l[elem]
        return h    

The reason for this heuristic is we want to minimize the wasted time crossing the bridge if the flashlight constraint was removed (if for example, it was actually daytime during the event).

But how does it work?

Since no one needs a flashlight while crossing the bridge, there's no reason for anyone to go back to help the others cross. And, we want to make the 2 people with the closest (or, just pretty close) travel time to cross the bridge together so that the time difference isn't too large.

For example, there are 4 people with travel time [1,2,5,10] respectively who want to cross the bridge. Only 2 people can be on the bridge at a time. Then, our heuristic function for this particular scenario is 10 + 2 = 12.

This is how we generate nodes. Since we sorted the list, we need a way to get its index from the original list. We use the orig attribute for this. The difference between generating nodes in Uniform Cost Search and A\* Search is we solve the heuristic and use that to get the evaluation_function of a node which will be used to compare the weights of the nodes.

In [22]:
def generate_nodes(self,node):
        fitness_len = len(self.fitness)
        if len(node.steps) % 3 == 0:
            h = self.find_heuristic(node,0)
            fin = []
            left = []
            c = 0
            for elem in range(fitness_len):
                    if node.state[elem] == 0:
                            left.append(elem)
                            c += 1
            a = left[0]
            b = left[1]           
            if c == 2:
                fin = [[a,b]]                
            else:
                c = left[-1]
                d = left[-2]
                fin = [[a,b],[a,c],[d,c]]
               
            c = 0
            for elem in fin:
                    new_state = list(node.state)
                    new_state[elem[0]] = 1
                    new_state[elem[1]] = 1
                    
                    new_steps = list(node.steps)
                    if(self.fitness[elem[0]]==self.fitness[elem[1]]):
                        for f in range(self.orig.index(self.fitness[elem[0]])+1,fitness_len):
                            if(self.fitness[elem[1]]==self.fitness[f]):
                                new_steps.extend([self.orig.index(self.fitness[elem[0]])+1, f+1])
                                break
                    else:
                        new_steps.extend([self.orig.index(self.fitness[elem[0]])+1, self.orig.index(self.fitness[elem[1]])+1])

                    new_cost = node.cost + max(self.fitness[elem[0]], self.fitness[elem[1]])
                    new_node = Node_A_star(new_cost, h , new_state, new_steps)
                    
                    if not self.has_repeated_states(new_node):
                            bisect.insort(self.fringe, new_node)
        else:
            h = self.find_heuristic(node,1)
            for elem in list(range(fitness_len)):
                if (node.state[elem] == 1):
                    new_state = list(node.state)
                    new_state[elem] = 0

                    new_steps = list(node.steps)
                    new_steps.append(self.orig.index(self.fitness[elem])+1)

                    new_cost = node.cost + self.fitness[elem]

                    new_node = Node_A_star(new_cost, h, new_state, new_steps)
                    
                    if not self.has_repeated_states(new_node):
                        bisect.insort(self.fringe, new_node)

Then finally, to call the actual A\* Search, we use this method which is exactly the same as with its Uniform Cost Search counterpart.

In [23]:
def a_star(self):
        num_visited = 0;
        fitness_len = len(self.fitness)
        while True:
            if not self.fringe:
                return 'Failure'

            # Explore Fringe
            node = self.fringe.pop(0)
            num_visited += 1
            if node.state == [1] * fitness_len:
                print(node.cost, num_visited)
                node.print_steps()
                return 'Success'

            self.generate_nodes(node)

# 2. Input and Output

## 2.1 Uniform Cost Search Algorithm

### 2.1.1 Input: 1, 2, 5, 10

In [24]:
# import time
# t = time.process_time()

inp = [1,2,5,10]
tree = Tree(inp)
tree.uniform_cost_search()

# elapsed_time = time.process_time() - t
# print(elapsed_time)

# Expected execution time is approx. 0.0011 seconds

17 27
1 2 ->
1 <-
3 4 ->
2 <-
1 2 ->


### 2.1.2 Input: 1, 2, 5, 10, 3, 4, 14, 18, 20, 50

In [25]:
# import time
# t = time.process_time()

inp = [1,2,5,10,3,4,14,18,20,50]
tree = Tree(inp)
tree.uniform_cost_search()

# elapsed_time = time.process_time() - t
# print(elapsed_time)

# Expected execution time is approx. 9.77 seconds

104 7123
1 2 ->
1 <-
3 4 ->
2 <-
1 2 ->
1 <-
7 8 ->
2 <-
1 2 ->
1 <-
9 10 ->
2 <-
1 2 ->
1 <-
1 5 ->
1 <-
1 6 ->


### 2.1.3 Input: 1, 2, 5, 10, 12, 17, 24, 21, 20, 20, 11, 33, 15, 19, 55

In [ ]:
# import time
# t = time.process_time()

inp = [1,2,5,10,12,17,24,21,20,20,11,33,15,19,55]
tree = Tree(inp)
tree.uniform_cost_search()

# elapsed_time = time.process_time() - t
# print(elapsed_time)

# Expected execution time is approx. 19.58 hours

## 2.2 A\* Search Algorithm

### 2.2.1 Input: 1, 2, 5, 10

In [29]:
# import time
# t = time.process_time()

inp = [1,2,5,10]

state_space = Tree_A_star(inp)
state_space.a_star()

#elapsed_time = time.process_time() - t
#print(elapsed_time)

# Expected execution time is approx. 0.00031 seconds

17 18
1 2 ->
1 <-
3 4 ->
2 <-
1 2 ->


'Success'

### 2.2.2 Input: 1, 2, 5, 10, 3, 4, 14, 18, 20, 50

In [30]:
# import time
# t = time.process_time()

inp = [1,2,5,10,3,4,14,18,20,50]

state_space = Tree_A_star(inp)
state_space.a_star()

#elapsed_time = time.process_time() - t
#print(elapsed_time)

# Expected execution time is approx. 0.06762 seconds

104 507
1 2 ->
1 <-
9 10 ->
2 <-
1 2 ->
1 <-
7 8 ->
2 <-
1 2 ->
1 <-
3 4 ->
2 <-
1 2 ->
1 <-
1 5 ->
1 <-
1 6 ->


'Success'

### 2.2.3 Input: 1, 2, 5, 10, 12, 17, 24, 21, 20, 20, 11, 33, 15, 19, 55

In [31]:
# import time
# t = time.process_time()

inp = [1,2,5,10,12,17,24,21,20,20,11,33,15,19,55]

state_space = Tree_A_star(inp)
state_space.a_star()

#elapsed_time = time.process_time() - t
#print(elapsed_time)

# Expected execution time is approx. 3.28096 seconds

182 3285
1 2 ->
1 <-
12 15 ->
2 <-
1 2 ->
1 <-
8 7 ->
2 <-
1 2 ->
1 <-
9 10 ->
2 <-
1 2 ->
1 <-
6 14 ->
2 <-
1 2 ->
1 <-
5 13 ->
2 <-
1 2 ->
1 <-
4 11 ->
2 <-
1 2 ->
1 <-
1 3 ->


'Success'

# 3. Analysis and Conclusion

An appropriate criteria for comparing the two algorithms is by analyzing their time and space complexities, and their running time on a couple of inputs. The complexities of each algorithm will only serve as a theoretical comparison. The more interesting finding would come from the actual running time of the code implementing the two algorithms.

## 3.1 Time and Space Complexity

The time and space complexity of the Uniform Cost Search algorithm is: $O\left(b^{floor({\frac{C}{\epsilon }})}\right)$

On the other hand, the time and space complexity of A* algorithm is: $O\left(b^d\right)$

The symbols are defined as follows: $b$ is the branching factor of the search tree, $d$ is the least-cost depth of the search tree (*i.e.* the shortest path from root to a leaf node), $C$ is destination cost, and each edge cost is atleast $\epsilon$.

Both of the algorithms have an exponential complexity with a common base of $b$. An analysis of the complexities between the two is rather noncomplex; a comparison between the exponential terms would suffice. The floor of $C/\epsilon$ is the maximal possible depth an algorithm must traverse to reach a goal state. Clearly, the A\* algorithm is generally and theoretically the faster algorithm since $d$ is a cheaper cost then the floor $C/\epsilon$.

As a conclusion, for all of the possible solvable problems, including the Bride Crossing Problem, A\* is the more efficient algorithm to use than the Uniform Cost Search. This is the theoretical standpoint of the problem, which would already suffice as an analysis and conclusion. However, the analysis of the problem majorly depends on the written code implementing the two algorithms.

## 3.2 Running Time of Code

As seen in Part 2 of this notebook, the two main code is tested with three different inputs of size 4, 10, and 15. Also, note that the execution time of the code is also specified in that Part written as a comment inside the code. As a context, each of the inputs were tested and timed using a 2.6 GHz Intel Core i5 machine.

The code for the Uniform Cost Search performed well for the input of size 4, accumulating an execution time of approximately 0.0011 seconds. The code also expanded 27 nodes which is relatively few. For the second input, with size 10, the code struggled a bit. It garnered an execution time of 9.77 seconds and expanded 7123 nodes. Lastly, for the input of size 15, the code massively struggled and was severely slow. It had an execution time of approximately 19.58 hours, expanding 335736 nodes.

The code for the A\* search performed exceptionally well for all three inputs. For the first input, the code had an execution time of 0.00031 seconds, expanding only 18 nodes. For the second input, it executed for about 0.06762 seconds and expanding 507 nodes. Lastly, for the third input, it accumulated an execution time of 3.280963285 seconds and expanded 3285 nodes.

For the analysis of the running time of the two code, a comparison using the *time percentage improvement formula* is essential. The formula is $\frac{\left(time_1\:-\:time_2\right)}{time_2}\cdot 100\%$, which will yield the improvement of $time_1$ with respect to $time_2$. For the first input, the code for A\* search accumulated a $71.82\%$ execution time improvement from the running time of the code for Uniform Cost Search. For the second input, A\* had an outstanding $99.32\%$ improvement. Lastly, for the third input, A\* had an excpetionally significant $99.995\%$ improvement.

As a conclusion, the performance of the code implementing the A\* search algorithm is significantly better than the performance of the Uniform Cost Search code. 

# 4. Individual Contributions